In [ ]:
from astropy.io import fits
import pandas as pd
import numpy as np

# Open the file without loading all data into memory
with fits.open('dr1_galaxy_stellarmass_lineinfo_v1.0.fits', memmap=True) as hdul:
    data = hdul['EMLINES_MASS'].data
    
    # Expanded list of columns to keep
    cols_to_keep = [
        # Identifiers and basic info
        'TARGETID', 'TARGET_RA', 'TARGET_DEC', 'HEALPIX', 'SURVEY', 'PROGRAM',
        
        # Redshift and spectroscopic quality
        'Z', 'ZERR', 'OBJTYPE', 'COADD_NUMEXP', 'SNR_MED',
        
        # Galaxy shape and morphology
        'SHAPE_R', 'SHAPE_R_IVAR',
        
        # CIGALE measurements
        'MASS_CG', 'MASSERR_CG', 'Z_CG', 'ZERR_CG', 'AGE_CG', 'AGEERR_CG', 
        'TAU_CG', 'TAUERR_CG', 'SFR_CG', 'SFRERR_CG', 'AV_CG', 'AVERR_CG',
        
        # STARLIGHT measurements
        'Z_SL', 'AGE_SL', 'AV_SL', 'MASSINI_SL', 'MASSCOR_SL', 
        'V0_SL', 'VD_SL', 'SNR_SL',
    ]
    
    # Create a dictionary for pandas DataFrame
    data_dict = {col: data[col] for col in cols_to_keep if col in data.columns}
    
    # Create DataFrame
    df = pd.DataFrame(data_dict)
    
    # Calculate some common color columns
    try:
        df['g_mag'] = -2.5 * np.log10(df['FLUX_G']) + 22.5
        df['r_mag'] = -2.5 * np.log10(df['FLUX_R']) + 22.5
        df['z_mag'] = -2.5 * np.log10(df['FLUX_Z']) + 22.5
        df['W1_mag'] = -2.5 * np.log10(df['FLUX_W1']) + 22.5
        df['W2_mag'] = -2.5 * np.log10(df['FLUX_W2']) + 22.5
        
        # Some useful colors
        df['g_r_color'] = df['g_mag'] - df['r_mag']
        df['r_z_color'] = df['r_mag'] - df['z_mag']
        df['W1_W2_color'] = df['W1_mag'] - df['W2_mag']
    except:
        print("Warning: Error calculating magnitude columns - check for zero or negative flux values")
    
    df.to_csv('../data/desi_catalogue.csv', index=False)